# Eager Execution

In [ ]:
import syft as sy
sy.requires(">=0.8.1-beta")

In [ ]:
node = sy.orchestra.launch(name="test-domain-1", port=8080, dev_mode=True)

In [ ]:
client = node.login(email="info@openmined.org", password="changethis")
client

In [ ]:
guest_client = node.client.guest()
guest_client

In [ ]:
assert client.credentials != guest_client.credentials

# Simple permissions

In [ ]:
import numpy as np

In [ ]:
input_obj = sy.TwinObject(
    private_obj=np.array([[3,3,3], [3,3,3]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

In [ ]:
# user creates
input_ptr = client.api.services.action.set(input_obj)

# guest gets pointer
pointer = guest_client.api.services.action.get_pointer(input_ptr.id)
pointer.id

In [ ]:
flat_ptr = pointer.flatten()
flat_ptr

In [ ]:
# read permission error
res_guest = guest_client.api.services.action.get(flat_ptr.id)
print(res_guest)
assert not isinstance(res_guest, sy.ActionObject)

In [ ]:
# get as user with permission
res_root = flat_ptr.get_from(client)
assert all(res_root == [3,3,3,3,3,3])

# Plans

## Plans with object instantiations

In [ ]:
@sy.planify
def my_plan(x=np.array([1,2,3,4,5,6])):
    return x+1

In [ ]:
plan_ptr = my_plan.send(guest_client)

In [ ]:
input_obj = sy.TwinObject(
    private_obj=np.array([1,2,3,4,5,6]),
    mock_obj=np.array([1,1,1,1,1,1])
)

In [ ]:
_id = client.api.services.action.set(input_obj).id
pointer = guest_client.api.services.action.get_pointer(_id) 

In [ ]:
res_ptr = plan_ptr(x=pointer)

In [ ]:
res_ptr.get_from(client)

## Complex plans

In [ ]:
@sy.planify
def my_plan(x=np.array([1,2,3,4,5,6])):
    y = x.flatten() # method -> [1,2,3,4,5,6]
    min_val = x.min() # method -> 1
    s = x.shape # getattribute -> 6    
    w = x[min_val] # __getitem__ -> 2
    y[min_val] = min_val # __setitem__ y = [1,1,3,4,5,6]
    res = client.api.lib.numpy.sum(y) + w + s # client function and __add__ 20 + 2 + 6 = 28
    return res #28

In [ ]:
my_plan

In [ ]:
plan_ptr = my_plan.send(guest_client)

In [ ]:
input_obj = sy.TwinObject(
    private_obj=np.array([1,2,3,4,5,6]),
    mock_obj=np.array([1,1,1, 1,1,1])
)

_id = client.api.services.action.set(input_obj).id
pointer = guest_client.api.services.action.get_pointer(_id) 

In [ ]:
res_ptr = plan_ptr(x=pointer)

In [ ]:
res_ptr.get_from(client)

## Plans with function calls

In [ ]:
client.api.lib.numpy.sum

In [ ]:
@sy.planify
def my_plan(x=np.array([[2,2,2], [2,2,2]])):
    y = x.flatten()
    w = client.api.lib.numpy.sum(y)
    return w

In [ ]:
plan_ptr = my_plan.send(guest_client)

input_obj = sy.TwinObject(
    private_obj=np.array([[3,3,3], [3,3,3]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

input_ptr = client.api.services.action.set(input_obj)
pointer = guest_client.api.services.action.get_pointer(input_ptr.id) 


In [ ]:
res_ptr = plan_ptr(x=pointer)

In [ ]:
assert res_ptr.get_from(client) == 18

## simple execution (root downloads)

In [ ]:
@sy.planify
def my_plan(x=np.array([[2,2,2], [2,2,2]])):
    y = x.flatten()
    z = y.prod()
    return z

In [ ]:
plan_ptr = my_plan.send(guest_client)

In [ ]:
input_obj = sy.TwinObject(
    private_obj=np.array([[3,3,3], [3,3,3]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

In [ ]:
input_ptr = client.api.services.action.set(input_obj)

In [ ]:
pointer = guest_client.api.services.action.get_pointer(input_ptr.id) 

In [ ]:
res_ptr = plan_ptr(x=pointer)

In [ ]:
# TODO: should be 1
res_ptr

In [ ]:
assert not isinstance(guest_client.api.services.action.get(res_ptr.id), sy.ActionObject)

In [ ]:
assert res_ptr.get_from(client) == np.array([[3,3,3], [3,3,3]]).flatten().prod()

## Downloading the result

In [ ]:
res_ptr.request(guest_client)

In [ ]:
# TODO: fix this issue with custom 

In [ ]:
client.api.services.request[0].approve_with_client(client)

In [ ]:
res_ptr.get_from(guest_client)

# Pointers (Twins)

## setattribute

In [ ]:
private_data, mock_data = np.array(
    [[1.0,2.0,3.0], [4.0,5.0,6.0]]), np.array([[1.0,1.0,1.0], [1.0,1.0,1.0]]
)

In [ ]:
obj = sy.TwinObject(
    private_obj=private_data,
    mock_obj=mock_data
)

In [ ]:
obj_pointer = client.api.services.action.set(obj)
obj_pointer = guest_client.api.services.action.get_pointer(obj_pointer.id)

In [ ]:
original_id = obj_pointer.id

In [ ]:
obj_pointer

In [ ]:
# note that this messes up the data and the shape of the array
obj_pointer.dtype = np.int32  

In [ ]:
res = obj_pointer.get_from(client)

In [ ]:
assert res.dtype == np.int32

In [ ]:
private_data.dtype= np.int32
mock_data.dtype= np.int32

In [ ]:
assert (res == private_data).all()
assert (obj_pointer.syft_action_data == mock_data).all()
assert not (obj_pointer.syft_action_data == private_data).all()

## getattribute

In [ ]:
obj = sy.TwinObject(
    private_obj=np.array([[1,2,3], [4,5,6]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

In [ ]:
obj_pointer = client.api.services.action.set(obj)
obj_pointer = guest_client.api.services.action.get_pointer(obj_pointer.id)

In [ ]:
size_pointer = obj_pointer.size   

In [ ]:
assert size_pointer.get_from(client) == 6

## methods

In [ ]:
obj = sy.TwinObject(
    private_obj=np.array([[1,2,3], [4,5,6]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

In [ ]:
obj_pointer = client.api.services.action.set(obj)
obj_pointer = guest_client.api.services.action.get_pointer(obj_pointer.id)

In [ ]:
flat_pointer = obj_pointer.flatten()   

In [ ]:
assert all(flat_pointer.get_from(client) == np.array([1,2,3,4,5,6]))

## Indexing

In [ ]:
obj = sy.TwinObject(
    private_obj=np.array([[1,2,3], [4,5,6]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

In [ ]:
obj_pointer = client.api.services.action.set(obj)
obj_pointer = guest_client.api.services.action.get_pointer(obj_pointer.id)

In [ ]:
indexed_pointer = obj_pointer[0, 0:2]   

In [ ]:
assert all(indexed_pointer.get_from(client) == np.array([1,2]))

In [ ]:
# if node.node_type.value == "python":
node.land()